# x86_64: Phase 1

In [1]:
# Imports

import angr
import claripy

Load the binary.

In [2]:
project = angr.Project('bomb64')

Define the address for the start of the execution path.

In [3]:
# The address where the symbolic execution shall begin. It is the beginning of the middle block where the key phrase is verified.
addr_start = 0x401604

# The address where the hook will be established.
# It is right before the string comparison function.
# Dumps it's arguments and finds out the reference string.
addr_hook = 0x40160E

addr_avoid = 0x401635

addr_target = 0x40162B

Define a blank state for the simulation, a state with most of it's data uninitialized. Pass the address where the state is initialized, along with the user input to be given via standard input.

In [4]:
state = project.factory.blank_state(addr=addr_start)

We define and add the hook to the function.

In [5]:
# Takes the state as initialiser.
class HookClass(angr.SimProcedure):
    def run(self):
        # Dumps the address of the argument to "strings_not_equal" function from the register.
        addr = self.state.regs.esi
        # Dereferences the address of the string and prints it.
        print(self.state.mem[addr].string.concrete)
        return 0

# Adds the hook to the execution flow.
project.hook(addr_hook, HookClass())

Create a simulation manager with this blank state that would help us manage the symbolic execution.

In [6]:
simgr = project.factory.simulation_manager(state)

Run the simulation manager so as to walk over various states.

In [7]:
simgr.explore(find=addr_target, avoid=addr_avoid)

WARNING | 2020-06-16 14:47:18,792 | angr.state_plugins.symbolic_memory | The program is accessing memory or registers with an unspecified value. This could indicate unwanted behavior.
WARNING | 2020-06-16 14:47:18,793 | angr.state_plugins.symbolic_memory | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING | 2020-06-16 14:47:18,794 | angr.state_plugins.symbolic_memory | 1) setting a value to the initial state
WARNING | 2020-06-16 14:47:18,794 | angr.state_plugins.symbolic_memory | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING | 2020-06-16 14:47:18,795 | angr.state_plugins.symbolic_memory | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY_REGISTERS}, to suppress these messages.
WARNING | 2020-06-16 14:47:18,796 | angr.state_plugins.symbolic_memory | Filling memory at 0x7fffffffffefff8 with 8 unconstrained bytes referenced from 0x40160e (p

b'DrEvil'


WARNING | 2020-06-16 14:47:19,117 | angr.engines.successors | Exit state has over 256 possible solutions. Likely unconstrained; skipping. <BV64 mem_7fffffffffefff8_39_64{UNINITIALIZED}>


<SimulationManager with 1 unconstrained>

There's the string we need to append to the input we provide to the input for ```phase_4```.